## Trading in Option Chain Data

1. Fetch historical data from Fyers API
2. Define the expiry and symbol for the option chain
3. Set the start date and end date for the historical data
4. Retrieve the historical data using the `get_fyers_data()` function
5. Check if the historical data is available
6. Implement a backtesting strategy using the historical data
7. Generate trade signals based on the strategy
8. Fetch the current price and strike price of the options from the option chain
9. Open positions based on the trade signals and option prices
10. Set an exit strategy to close positions if certain conditions are met
11. Calculate the profit/loss for each position
12. Convert the positions to a DataFrame for analysis
13. Visualize the trading results using charts and graphs
14. Analyze the performance of the trading strategy
15. Make adjustments to the strategy based on the analysis
16. Repeat the process for different expiry dates and symbols
17. Monitor the market conditions and adjust the strategy accordingly
18. Continuously evaluate and improve the trading strategy
```




### Installing Python Libraries

In [ ]:
pip install pandas numpy mplfinance matplotlib ta fyers-apiv3 pandas_ta

<!-- Import Libraries -->

In [11]:
from fyers_api import accessToken
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import webbrowser
import pandas as pd
import time
import numpy as np
import math
from datetime import date
import mplfinance as mpf
from ta.trend import MACD
import matplotlib.pyplot as plt
import ta
from urllib.parse import urlparse, parse_qs

In [12]:
# login process

client_id = "VVCFTCB9Y7-100"
secret_key = "8U5LQ172V9"
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"
response_type = "code"
grant_type = "authorization_code"
emadata = pd.DataFrame()
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type,
    grant_type=grant_type,
)

In [14]:
response = session.generate_authcode()
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=VVCFTCB9Y7-100&redirect_uri=https%3A%2F%2Ftrade.fyers.in%2Fapi-login%2Fredirect-uri%2Findex.html&response_type=code&state=None


In [15]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MTUwMzU5NTcsImV4cCI6MTcxNTA2NTk1NywibmJmIjoxNzE1MDM1MzU3LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJZSzA3MDk2Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiYmViODQyNjI5ODcxMmNhOThjNTY2NDk2MWFjMTdjMDM3ZGQ0OWFiYWJjMzE1ZDY1YWE4ZDEyZDAiLCJub25jZSI6IiIsImFwcF9pZCI6IlZWQ0ZUQ0I5WTciLCJ1dWlkIjoiNGZhNzU5ZTAwMzhiNGZhODgxNmMyNTA3MjZiNzg5MGYiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.pMNWJonMK8a1I62oTh1D3hfb9L5I3cTLonkD8EQnqro"

In [16]:
# Set the authorization code in the session object
session.set_token(auth_code)
# Generate the access token using the authorization code
response = session.generate_token()
# Print the response, which should contain the access token and other details
print(response)

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MTUwMzU5NzksImV4cCI6MTcxNTA0MTg1OSwibmJmIjoxNzE1MDM1OTc5LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbU9WOUx4RkVwX1NEVHRqXzMtMU9HTWpOdm84WFZ3TVYwYlU4c2hmZ1pSUlpBeEdiQXhwdFZzcWJuNGwtaTM0TjFtNUpfWEpJc3IzM0FqZGxxUFVhUXJ3WWlpcGlYVFdmQUhpTGlFcVdXZVVLcjd0VT0iLCJkaXNwbGF5X25hbWUiOiJLQU1BTEEgIiwib21zIjoiSzEiLCJoc21fa2V5IjoiYmViODQyNjI5ODcxMmNhOThjNTY2NDk2MWFjMTdjMDM3ZGQ0OWFiYWJjMzE1ZDY1YWE4ZDEyZDAiLCJmeV9pZCI6IllLMDcwOTYiLCJhcHBUeXBlIjoxMDAsInBvYV9mbGFnIjoiTiJ9.pthjPvZHgMwYw7uFBc0iMycTbp3qGUNgH1S6mjgLk48', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MTUwMzU5NzksImV4cCI6MTcxNjI1MTQ1OSwibmJmIjoxNzE1MDM1OTc5LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQm1PVjlMeEZFcF9T

In [17]:
df = pd.DataFrame()
df = response
tk = df["access_token"]
# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=tk, log_path=""
)

In [18]:
import sys

sys.path.append("F:/GitHub/pythonenv/PythonEnvProjects")
from PositionManager import PositionManager

# Initialize PositionManager instance


position_manager = PositionManager()

In [19]:
# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=tk, log_path="C:\\fyersdata"
)
responseValidate = fyers.get_profile()
print(responseValidate)

{'s': 'ok', 'code': 200, 'message': '', 'data': {'fy_id': 'YK07096', 'name': 'KAMALA ', 'image': None, 'display_name': None, 'pin_change_date': '05-05-2024 19:12:55', 'email_id': 'gsk.shivu5@gmail.com', 'pwd_change_date': None, 'PAN': '---------', 'mobile_number': '9741564530', 'totp': False, 'pwd_to_expire': 90}}


In [20]:
# Fetch Historical Data of the Index or Stocks
def get_fyers_data(expiry, symbol, start_date, end_date):

    try:

        # Get historical data

        cdata = {
            "expiry": expiry,
            "symbol": symbol,
            "resolution": "1",
            "date_format": "1",
            "range_from": start_date,
            "range_to": end_date,
        }

        response = fyers.history(data=cdata)

        data = pd.DataFrame.from_dict(response["candles"])

        cols = ["Datetime", "Open", "High", "Low", "Close", "Volume"]

        data.columns = cols

        data["Datetime"] = pd.to_datetime(data["Datetime"], unit="s")

        data["Datetime"] = (
            data["Datetime"].dt.tz_localize("UTC").dt.tz_convert("Asia/Kolkata")
        ).dt.tz_localize(None)

        data = data.set_index("Datetime")


        # Drop duplicate index values


        data = data.loc[~data.index.duplicated(keep="first")]


        return data


    except Exception as e:

        print(e.message)

        return None

In [ ]:
# Fetch Historical Option Data of the Index or Stocks
def get_fyers_option_data(expiry, symbol, start_date, end_date):

    try:
        

In [21]:
expiry = ""
symbol = "NSE:NIFTY50-INDEX"
start_date = "2024-04-03"
end_date = "2024-05-07"

In [43]:
import pandas as pd

cdata = {"symbol": symbol, "strikecount": 2, "timestamp": ""}

# update values to the variables
response = fyers.optionchain(data=cdata)
expiry = response["data"]["expiryData"][0]["date"]


# Get the option chain data
option_data = response["data"]["optionsChain"]

# Convert the list of dictionaries to a DataFrame
option_data = pd.DataFrame(option_data)
print(option_data)

       ask     bid    description ex_symbol exchange       fp  fpch  fpchp  \
0     0.00    0.00  NIFTY50-INDEX     NIFTY      NSE  22563.3 -11.9  -0.05   
1    69.15   68.05            NaN       NaN      NaN      NaN   NaN    NaN   
2   200.00  198.00            NaN       NaN      NaN      NaN   NaN    NaN   
3   167.90  165.00            NaN       NaN      NaN      NaN   NaN    NaN   
4    86.60   85.00            NaN       NaN      NaN      NaN   NaN    NaN   
5   108.35  107.00            NaN       NaN      NaN      NaN   NaN    NaN   
6   139.35  137.45            NaN       NaN      NaN      NaN   NaN    NaN   
7   113.50  112.50            NaN       NaN      NaN      NaN   NaN    NaN   
8   132.45  131.10            NaN       NaN      NaN      NaN   NaN    NaN   
9   159.95  158.10            NaN       NaN      NaN      NaN   NaN    NaN   
10   91.00   89.95            NaN       NaN      NaN      NaN   NaN    NaN   

            fyToken       ltp  ltpch  ltpchp option_type  strik

KeyError: 'strikePrice'

In [ ]:
def long_options_strategy(data):
    # Reset index to ensure unique values
    data = data.reset_index(drop=True)

    # Calculate MACD with fast EMA of 9 periods and slow EMA of 13 periods
    macd = data.ta.macd(fast=9, slow=13)
    data["MACD"] = macd["MACD_9_13_9"]
    data["Signal"] = macd["MACDs_9_13_9"]

    print(data[["MACD", "Signal"]])  # Print MACD and Signal values

    # Generate entry and exit signals
    entry_points_call = []
    entry_points_put = []

    for i in range(1, len(data)):
        # Entry condition for Long Call
        if data["MACD"].iloc[i] > data["Signal"].iloc[i]:
            entry_points_call.append(i)

        # Entry condition for Long Put
        if data["MACD"].iloc[i] < data["Signal"].iloc[i]:
            entry_points_put.append(i)

    return entry_points_call, entry_points_put

In [ ]:
# def get_option_chain(symbol, expiry):
#     data = {"symbol": symbol, "strikecount": 1, "timestamp": expiry}
#     response = fyers.optionchain(data=data)

#     # Fetch the current price of the underlying asset
#     current_price = next(
#         (item for item in response["data"]["optionsChain"] if item["symbol"] == symbol),
#         None,
#     )
#     if current_price is None:
#         raise ValueError(f"Could not find current price for symbol {symbol}")
#     current_price = current_price["ltp"]

#     # Fetch the option chain
#     option_chain = {"CE": [], "PE": []}
#     for item in response["data"]["optionsChain"]:
#         if item["option_type"] == "CE":
#             option_chain["CE"].append(item)
#         elif item["option_type"] == "PE":
#             option_chain["PE"].append(item)

#     # Find the call and put options with strike prices closest to the current price
#     call_option = min(
#         option_chain["CE"], key=lambda x: abs(x["strikePrice"] - current_price)
#     )
#     put_option = min(
#         option_chain["PE"], key=lambda x: abs(x["strikePrice"] - current_price)
#     )

#     return current_price, call_option, put_option

In [ ]:
def get_atm_option_price(symbol, expiry, historical_data, index_price, position_type):
    # Fetch the option chain from the historical data
    option_chain = historical_data[
        (historical_data["symbol"] == symbol) & (historical_data["expiry"] == expiry)
    ]

    # Filter the options based on the position type
    options = option_chain[option_chain["option_type"] == position_type]

    # Find the option with strike price closest to the index price
    atm_option = min(
        options,
        key=lambda x: abs(x["strikePrice"] - index_price),
    )

    return atm_option["close"]

In [ ]:
# def backtest_strategy(data, symbol, expiry):
#     # Add 9-period and 5-period exponential moving averages (EMA)
#     data["ema_9"] = data["Close"].ewm(span=9, adjust=False).mean()
#     data["ema_5"] = data["Close"].ewm(span=5, adjust=False).mean()

#     # Generate trade signals
#     entry_points_call, entry_points_put = long_options_strategy(data)

#     # Backtest
#     positions = []
#     for i in range(len(data)):
#         if data.index[i] in entry_points_call:
#             # Fetch the current price and the strike price of the call option
#             current_price, call_option, _ = get_option_chain(symbol, expiry)

#             position = position_manager.open_position(
#                 "Call", call_option["strikePrice"], data.index[i], "Long Call"
#             )
#             positions.append(position)
#         elif data.index[i] in entry_points_put:
#             # Fetch the current price and the strike price of the put option
#             current_price, _, put_option = get_option_chain(symbol, expiry)

#             position = position_manager.open_position(
#                 "Put", put_option["strikePrice"], data.index[i], "Long Put"
#             )
#             positions.append(position)

#         # Exit strategy: close the position if the price closes below the 5-period EMA
#         if len(positions) > 0 and data["Close"].iloc[i] < data["ema_5"].iloc[i]:
#             position_manager.close_position(0, data["Close"].iloc[i], data.index[i])

#     return positions

In [ ]:
class Position:
    def __init__(self, position_type, entry_price, entry_time, symbol):
        self.position_type = position_type
        self.entry_price = entry_price
        self.entry_time = entry_time
        self.symbol = symbol
        self.exit_price = None
        self.exit_time = None

    def close_position(self, price, time):
        self.exit_price = price
        self.exit_time = time

In [ ]:
def backtest_strategy(ohlc, symbol, expiry):
    positions = []
    ema_short = ohlc["Close"].ewm(span=12, adjust=False).mean()
    ema_long = ohlc["Close"].ewm(span=26, adjust=False).mean()

    for i in range(1, len(ohlc)):
        if (
            ema_short.iloc[i] > ema_long.iloc[i]
            and ema_short.iloc[i - 1] < ema_long.iloc[i - 1]
        ):
            option_price = get_atm_option_price(ohlc["Close"].iloc[i], "Long")
            positions.append(Position("Long", option_price, ohlc.index[i], symbol))

        elif (
            ema_short.iloc[i] < ema_long.iloc[i]
            and ema_short.iloc[i - 1] > ema_long.iloc[i - 1]
        ):
            option_price = get_atm_option_price(ohlc["Close"].iloc[i], "Short")
            positions.append(Position("Short", option_price, ohlc.index[i], symbol))

    for position in positions:
        if position.exit_price is None:
            position.close_position(ohlc["Close"].iloc[-1], ohlc.index[-1])

    return positions

In [ ]:
def main():
    try:
        # Fetch historical data from Fyers API
        expiry = "NSE:NIFTY24509"
        symbol = "NSE:NIFTY50-INDEX"
        start_date = "2024-04-03"
        end_date = "2024-05-07"
        ohlc = get_fyers_data(expiry, symbol, start_date, end_date)

        if ohlc is not None:
            # Backtest strategy
            positions = backtest_strategy(ohlc, symbol, expiry)

            # Convert positions to a DataFrame
            data = []
            for position in positions:
                profit_loss = position.calculate_profit_loss()
                data.append(
                    {
                        "Position Type": position.position_type,
                        "Entry Price": position.entry_price,
                        "Entry Time": position.entry_time,
                        "Exit Price": position.exit_price,
                        "Profit/Loss": profit_loss,
                    }
                )
            df = pd.DataFrame(data)

            # Export DataFrame to a CSV file
            df.to_csv("backtest_results.csv", index=False)

            # Print trade information
            for position in positions:
                profit_loss = position.calculate_profit_loss()
                print(
                    f"Position Type: {position.position_type}, Entry Price: {position.entry_price}, Entry Time: {position.entry_time}, Exit Price: {position.exit_price}, Profit/Loss: {profit_loss}"
                )
            # Plot candlestick chart with entry and exit points
            mpf.plot(
                ohlc,
                type="candle",
                volume=False,
                style="yahoo",
                title="Nifty50",
                ylabel="Price",
                addplot=[
                    mpf.make_addplot(ohlc["MACD"]),
                    mpf.make_addplot(ohlc["Signal"]),
                    mpf.make_addplot(
                        ohlc.loc[positions.index]["Close"],
                        scatter=True,
                        markersize=100,
                        marker="x",
                        color="r",
                    ),
                ],
                figscale=2,
                figsize=(15, 8),
            )

        else:
            print("Failed to fetch data from Fyers API")
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()

In [ ]:
# import matplotlib.pyplot as plt
# import pandas_ta as ta
# import pandas as pd

# def plot_macd(data):
#     # Reset index to ensure unique values
#     data = data.reset_index(drop=True)

#     # Calculate MACD
#     macd = data.ta.macd(close="Close")
#     data["MACD"] = macd["MACD_12_26_9"]
#     data["Signal"] = macd["MACDs_12_26_9"]

#     # Create signals
#     data['Buy_Signal'] = (data["MACD"] > data["Signal"])
#     data['Sell_Signal'] = (data["MACD"] < data["Signal"])

#     print(data[["MACD", "Signal", 'Buy_Signal', 'Sell_Signal']])  # Print MACD, Signal and trading signals

#     # Create a new figure
#     plt.figure(figsize=(12, 5))

#     # Plot MACD and Signal line
#     plt.plot(data.index, data["MACD"], label="MACD")
#     plt.plot(data.index, data["Signal"], label="Signal")

#     # Add a legend
#     plt.legend(loc="upper left")

#     # Show the plot
#     plt.show()

# # Fetch historical data from Fyers API
# expiry = "NSE:NIFTY24509"
# symbol = "NSE:NIFTY50-INDEX"
# start_date = "2024-03-01"  # Fetch data from a month earlier
# end_date = "2024-05-07"
# ohlc = get_fyers_data(expiry, symbol, start_date, end_date)

# if ohlc is not None:
#     print(ohlc)  # Print historical data
#     plot_macd(ohlc)

In [ ]:
# def plot_price(data):
#     # Create a new figure
#     plt.figure(figsize=(12, 5))

#     # Plot closing price
#     plt.plot(data.index, data["Close"], label="Close")

#     # Add a legend
#     plt.legend(loc='upper left')

#     # Show the plot
#     plt.show()

# # Fetch historical data from Fyers API
# expiry = "NSE:NIFTY24509"
# symbol = "NSE:NIFTY50-INDEX"
# start_date = "2024-03-01"  # Fetch data from a month earlier
# end_date = "2024-05-07"
# ohlc = get_fyers_data(expiry, symbol, start_date, end_date)

# if ohlc is not None:
#     print(ohlc)  # Print historical data
#     plot_price(ohlc)

In [ ]:
# import mplfinance as mpf
# import pandas_ta as ta

# def plot_price(data):
#     # Drop duplicate index values
#     data = data.loc[~data.index.duplicated(keep='first')]

#     # Calculate MACD
#     macd = ta.macd(data["Close"])

#     # Create a candlestick chart
#     ap = [mpf.make_addplot(macd['MACD_12_26_9']), mpf.make_addplot(macd['MACDs_12_26_9'])]
#     mpf.plot(data, type='candle', style='charles', title='Candlestick chart', ylabel='Price', addplot=ap)

# # Fetch historical data from Fyers API
# expiry = "NSE:NIFTY24509"
# symbol = "NSE:NIFTY50-INDEX"
# start_date = "2024-03-01"  # Fetch data from a month earlier
# end_date = "2024-05-07"
# ohlc = get_fyers_data(expiry, symbol, start_date, end_date)

# if ohlc is not None:
#     print(ohlc)  # Print historical data
#     plot_price(ohlc)

In [ ]:
# import mplfinance as mpf
# import pandas_ta as ta

# def plot_price(data):
#     # Drop duplicate index values
#     data = data.loc[~data.index.duplicated(keep='first')]

#     # Calculate MACD
#     macd = ta.macd(data["Close"])

#     # Create signals
#     data['Buy_Signal'] = (macd['MACD_12_26_9'] > macd['MACDs_12_26_9'])
#     data['Sell_Signal'] = (macd['MACD_12_26_9'] < macd['MACDs_12_26_9'])

#     # Create a candlestick chart
#     ap = [mpf.make_addplot(macd['MACD_12_26_9']), mpf.make_addplot(macd['MACDs_12_26_9'])]
#     mpf.plot(data, type='candle', style='charles', title='Candlestick chart', ylabel='Price', addplot=ap)

#     # Print signals
#     print(data[['Buy_Signal', 'Sell_Signal']])

# # Fetch historical data from Fyers API
# expiry = "NSE:NIFTY24509"
# symbol = "NSE:NIFTY50-INDEX"
# start_date = "2024-03-01"  # Fetch data from a month earlier
# end_date = "2024-05-07"
# ohlc = get_fyers_data(expiry, symbol, start_date, end_date)

# if ohlc is not None:
#     print(ohlc)  # Print historical data
#     plot_price(ohlc)